In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 58.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 7.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 5.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57

In [3]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name())

CUDA Available: True
Device Name: Tesla P100-PCIE-16GB


In [4]:
import os
os.environ["WANDB_API_KEY"] = "af42a03cea75fc8b61985bca0977043b6de81fbb"

In [5]:
!pip install wandb

In [6]:
print("\nAttempting Weights & Biases login (should use environment variable)...")
import wandb
wandb.login()


Attempting Weights & Biases login (should use environment variable)...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: zyrogx (zyrogx-zyrogx-solutions) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
from datasets import load_dataset, DatasetDict

# Load the dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en")

# Format: Combine into <think>...</think> and <response>...</response> format
def format_example(example):
    return {
        "text": f"<s>[INST] {example['Question']} [/INST] <think>{example['Complex_CoT']}</think>\n<response>{example['Response']}</response>"
    }

# Apply formatting
formatted_dataset = dataset["train"].map(format_example)

README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

Map:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [8]:
dataset['train'][0]

{'Question': 'Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?',
 'Complex_CoT': "Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the l

In [9]:
formatted_dataset['text'][0]

"<s>[INST] Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings? [/INST] <think>Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the left sid

In [10]:
# Split dataset into train (19604) and validation (100)
train_dataset = formatted_dataset.select(range(19604))
val_dataset = formatted_dataset.select(range(19604, 19704))

# Final wrapped dataset
final_dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

# Check a sample
print(final_dataset["train"][0]["text"])

<s>[INST] Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings? [/INST] <think>Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?

But wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.

So, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?

Oh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.

Let's put this together: if a blood clot from the leg somehow travels to the left side of the 

In [11]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = 4096,
    dtype = torch.float16,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-23 11:05:42.802301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747998342.985950      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747998343.034655      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [12]:
# Tokenize using the unsloth tokenizer
tokenized_dataset = final_dataset.map(
    lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=4096),
    batched=True
)

Map:   0%|          | 0/19604 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:

from unsloth import FastLanguageModel # Ensure it's imported if in a new session
# model and tokenizer should already be loaded from Cell 11

# FastLanguageModel.for_inference(model)  # <<< REMOVE OR COMMENT OUT THIS LINE

model = FastLanguageModel.get_peft_model(
    model,
    r = 8,  # LoRA rank — balances capacity vs memory
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    # use_gradient_checkpointing is automatically handled well by Unsloth when True or "unsloth"
    # and is generally enabled by default for LoRA models if appropriate.
    # You can explicitly pass it if you want to be sure:
    use_gradient_checkpointing = "unsloth", # Or True
    random_state = 42,
    # max_seq_length is primarily a concern during model loading (FastLanguageModel.from_pretrained)
    # and tokenizer configuration, not directly in get_peft_model.
)

# The SFTTrainer initialization in Cell 18 should then work.

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.5.7 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [14]:
import transformers
print(transformers.__version__)

4.51.3


In [15]:
from transformers import TrainingArguments
print(TrainingArguments.__module__)

transformers.training_args


In [16]:
from transformers.training_args import TrainingArguments
import inspect

# Check where it's coming from
print("TrainingArguments class from:", TrainingArguments.__module__)
print("File path:", inspect.getfile(TrainingArguments))

# Print all arguments in __init__
print("\nSupported args in __init__:")
print(inspect.signature(TrainingArguments.__init__))

TrainingArguments class from: transformers.training_args
File path: /usr/local/lib/python3.11/dist-packages/transformers/training_args.py

Supported args in __init__:
(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_ty

In [17]:
from transformers.training_args import TrainingArguments


training_args = TrainingArguments(
    output_dir = "llama-medical-lora",   # Directory to save LoRA checkpoints
    num_train_epochs = 1,                # Start with 1 for test run; use 3–5 in real run
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    gradient_accumulation_steps = 4,     # Effective batch size = 8
    eval_strategy = "steps",
    eval_steps = 10,                     # Evaluate every 10 steps
    logging_steps = 5,
    save_strategy = "steps",
    save_steps = 50,
    learning_rate = 2e-4,                # Works well with LoRA
    fp16 = True,                         # Enable half-precision
    bf16 = False,                        # Set True if using bfloat16 hardware (like A100)
    max_steps = 200,                     # Set to None if using num_train_epochs
    warmup_steps = 10,
    report_to = "wandb",                 # Or use "none" if you're not using wandb
    logging_dir = "./logs",
    save_total_limit = 2,                # Keep only last 2 checkpoints
    load_best_model_at_end = False
)

In [18]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    packing = False,
    args=training_args,
    dataset_text_field="text",  # or whatever your text field is
)

In [ ]:
trainer.train()


In [21]:
# Create a new code cell after interrupting the kernel

# Option 1: Using the trainer object (recommended)
# This saves the adapter model, tokenizer, and training arguments.
desired_save_directory = "/kaggle/working/llama-medical-lora"
trainer.save_model(desired_save_directory)
print(f"Model (adapter) saved to {desired_save_directory}")

# Option 2: Saving the PEFT model and tokenizer directly
# model.save_pretrained(desired_save_directory)
# tokenizer.save_pretrained(desired_save_directory)
# print(f"Model (adapter) and tokenizer saved to {desired_save_directory}")

Model (adapter) saved to /kaggle/working/llama-medical-lora


In [24]:
from unsloth import FastLanguageModel
import torch
from peft import PeftModel # Import PeftModel

# 1. Define Model and Saved Adapter Information
base_model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
adapter_path = "/kaggle/working/llama-medical-lora"  # Or your specific step30 path like "llama-medical-lora-step30"
max_seq_length = 4096
dtype = torch.float16
load_in_4bit = True

# 2. Load the Base Model with Unsloth
print(f"Loading base model: {base_model_name}")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("Base model loaded.")

# 3. Load the LoRA Adapter onto the Base Model
print(f"Loading LoRA adapter from: {adapter_path}")
# For Unsloth, after loading the base model, you apply the adapter like this:
# The PeftModel class is used to load adapters onto an existing model.
# Unsloth models are compatible with this.
model = PeftModel.from_pretrained(
    model=model,  # Pass the already loaded Unsloth FastLanguageModel object
    model_id=adapter_path, # Pass the path to your adapter directory
)
print("LoRA adapter loaded.")

# 4. Prepare the model for inference
FastLanguageModel.for_inference(model)
print("Model prepared for inference.")

# 5. Define your prompt formatting function
def format_prompt_for_inference(question):
    return f"<s>[INST] {question.strip()} [/INST] "

# 6. Generate Output
your_medical_question = "The patient presents with a persistent dry cough, low-grade fever for the past 5 days, and mild body aches. What are the potential differential diagnoses and initial recommended steps?"
prompt = format_prompt_for_inference(your_medical_question)

inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=max_seq_length).to("cuda")

from transformers import GenerationConfig
generation_config = GenerationConfig(
    max_new_tokens=512,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print("\nGenerating response...")
with torch.no_grad():
    outputs = model.generate(**inputs, generation_config=generation_config)

generated_ids = outputs[0, inputs['input_ids'].shape[1]:]
response_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

print("\nModel's Response:")
print(response_text)

Loading base model: unsloth/Llama-3.2-3B-Instruct-bnb-4bit
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Base model loaded.
Loading LoRA adapter from: /kaggle/working/llama-medical-lora
LoRA adapter loaded.
Model prepared for inference.

Generating response...

Model's Response:
 <s>[Response]

The patient's symptoms suggest an infectious process, particularly respiratory in nature. Potential differential diagnoses include viral respiratory infections such as influenza or respiratory syncytial virus (RSV), as well as bacterial infections like Streptococcus pneumoniae or Haemoph

In [29]:
!pip install evaluate
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=335e1b0233b1cac1976d54f981d5b3a92b401563b73e643eab29cd14776aef75
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [30]:
# In a new cell, after training is done and you've loaded your chosen fine-tuned model/adapter
import evaluate
from tqdm import tqdm # For progress bar

rouge = evaluate.load('rouge')
model.eval() # Set model to evaluation mode

# Assuming 'model' and 'tokenizer' are your loaded fine-tuned model and its tokenizer
# And 'val_dataset' is your prepared validation set with 'original_question' and 'ground_truth_completion'

predictions_after_ft = []
references_after_ft = []

# Make sure tokenizer.pad_token_id is set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Define your prompt function for inference (should match training input part)
def format_prompt_for_eval(question):
    return f"<s>[INST] {question.strip()} [/INST] "

for example in tqdm(val_dataset): # val_dataset is your Dataset object
    question = example['text'].split("[/INST]")[0] + "[/INST] " # Extract the prompt part
    # Or, if you saved original_question separately in val_dataset:
    # question = format_prompt_for_eval(example['original_question'])

    ground_truth = example['text'].split("[/INST]")[1].strip() # Extract the target completion
    # Or, if you saved ground_truth_completion separately:
    # ground_truth = example['ground_truth_completion']


    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=4096-512).to("cuda") # Allow space for generation

    with torch.no_grad():
        # Adjust generation parameters as needed, make them consistent
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False, # For evaluation, often better to be deterministic
            num_beams=1 # Greedy decoding
        )

    # Decode only the generated part
    generated_text_ids = generated_ids[0, inputs['input_ids'].shape[1]:]
    prediction = tokenizer.decode(generated_text_ids, skip_special_tokens=True)

    predictions_after_ft.append(prediction)
    references_after_ft.append(ground_truth)

if predictions_after_ft and references_after_ft:
    results_after_ft = rouge.compute(predictions=predictions_after_ft, references=references_after_ft)
    print("ROUGE-L Score AFTER Fine-Tuning:", results_after_ft)
    if wandb.run: # If wandb run is still active or you re-init
         wandb.log({"rouge_L_after_finetuning": results_after_ft.get('rougeL', 0.0)})
else:
    print("Could not compute ROUGE-L (no predictions/references).")

100%|██████████| 100/100 [27:16<00:00, 16.36s/it]


ROUGE-L Score AFTER Fine-Tuning: {'rouge1': 0.33191925927800325, 'rouge2': 0.11320696374686806, 'rougeL': 0.17727151057759394, 'rougeLsum': 0.29601228585448774}


In [31]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN")
    login(token=hf_token)
    print("Successfully logged into Hugging Face Hub!")
except Exception as e:
    print(f"Hugging Face login failed. Ensure 'HF_TOKEN' is set as a Kaggle Secret: {e}")
    print("Alternatively, you can try a manual login cell (see next option).")

Successfully logged into Hugging Face Hub!


In [32]:

hf_model_repo_id = "zyrogX/Llama-3.2-3B-Instruct-medical-CoT-lora"
print(f"Will attempt to upload to: {hf_model_repo_id}")

Will attempt to upload to: zyrogX/Llama-3.2-3B-Instruct-medical-CoT-lora


In [33]:
adapter_checkpoint_path = "/kaggle/working/llama-medical-lora" # Or "/kaggle/working/llama-medical-lora/checkpoint-200" or whatever you named it

In [34]:
from unsloth import FastLanguageModel
import torch

base_model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
max_seq_length = 4096
dtype = torch.float16
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print(f"Base model '{base_model_name}' and tokenizer loaded.")

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Base model 'unsloth/Llama-3.2-3B-Instruct-bnb-4bit' and tokenizer loaded.


In [35]:
from peft import PeftModel

model = PeftModel.from_pretrained(
    model=model,
    model_id=adapter_checkpoint_path
)
# The tokenizer was also saved in adapter_checkpoint_path if you used trainer.save_model()
# So, you can also reload it from there to be absolutely sure it matches the adapter:
tokenizer = tokenizer.from_pretrained(adapter_checkpoint_path)

print(f"Fine-tuned LoRA adapter and tokenizer loaded from: {adapter_checkpoint_path}")

Fine-tuned LoRA adapter and tokenizer loaded from: /kaggle/working/llama-medical-lora


In [36]:
# Ensure 'model' is your fine-tuned PEFT model object
# Ensure 'tokenizer' is the associated tokenizer

try:
    print(f"Pushing LoRA adapter to {hf_model_repo_id}...")
    model.push_to_hub(
        repo_id=hf_model_repo_id,
        commit_message="Upload fine-tuned LoRA adapter (e.g., after 200 steps)" # Be specific
    )
    print("LoRA adapter pushed successfully!")

    print(f"\nPushing tokenizer to {hf_model_repo_id}...")
    tokenizer.push_to_hub(
        repo_id=hf_model_repo_id,
        commit_message="Upload tokenizer"
    )
    print("Tokenizer pushed successfully!")

    print(f"\nVisit your model page at: https://huggingface.co/{hf_model_repo_id}")

except Exception as e:
    print(f"An error occurred during upload: {e}")
    # Add troubleshooting tips here if needed

Pushing LoRA adapter to zyrogX/Llama-3.2-3B-Instruct-medical-CoT-lora...


adapter_model.safetensors:   0%|          | 0.00/48.7M [00:00<?, ?B/s]

LoRA adapter pushed successfully!

Pushing tokenizer to zyrogX/Llama-3.2-3B-Instruct-medical-CoT-lora...


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Tokenizer pushed successfully!

Visit your model page at: https://huggingface.co/zyrogX/Llama-3.2-3B-Instruct-medical-CoT-lora
